In [45]:
import mne
import os
from mne.time_frequency import psd_array_multitaper
import numpy as np
import pandas as pd
from scipy.stats import entropy, skew, kurtosis
import csv



In [46]:
# field names 
fields = ['File Name', 'Start', 'End'] 

# data rows of csv file 
rows = [['chb08_02.edf',	2670 ,	2841],
        ['chb08_05.edf',	2856 ,	3046 ],
        ['chb08_11.edf',	2988 ,	3122 ],
        ['chb08_13.edf',	2417 ,	2577 ],
        ['chb08_21.edf',	2083 ,	2347 ],
]


# name of csv file 
filename = "chb08_seizure.csv"

# writing to csv file 
with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(fields) 

        # writing the data rows 
        csvwriter.writerows(rows)

In [47]:
freq_bands = {'delta': [0.5, 4], 'theta': [4, 7], 'alpha': [7, 15], 'beta': [15, 30]}

In [66]:
def read_eeg_files(directory):
    eeg_data = []

    for filename in os.listdir(directory):
        if filename.endswith('.edf'):
            filepath = os.path.join(directory, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            eeg_data.append(raw)

    return eeg_data

def extract_spectral_features(raw, freq_bands):
    spectral_features = []
    # raw=np.array(raw)
    raw.filter(1, 120)
    print("********************************************")
    print(type(raw))
    raw.set_eeg_reference('average')
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    psds, freqs = psd_array_multitaper(raw.get_data()[eeg_picks], sfreq=raw.info['sfreq'], fmin=0.5, fmax=30) # type: ignore

    for band, freq_range in freq_bands.items():
        band_data = raw.copy().filter(freq_range[0], freq_range[1], picks=eeg_picks).get_data()

        if np.isnan(band_data).any():
            print(f"No data available for {band} band.")
            continue

        band_power = np.mean(np.abs(band_data) ** 2, axis=1)
        band_entropy = entropy(np.abs(band_data) ** 2, axis=1)
        band_skewness = skew(band_data, axis=1)

        spectral_features += [band_power, band_entropy, band_skewness]


    return spectral_features

def extract_time_features(raw):
    time_features = []

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    eeg_data = raw.get_data()[eeg_picks]

    for channel_data in eeg_data:
        channel_mean = np.mean(channel_data)

        channel_std = np.std(channel_data)

        channel_kurtosis = kurtosis(channel_data)
            
        channel_ptp = np.ptp(channel_data,axis=-1)

        channel_var = np.var(channel_data,axis=-1)

        channel_min = np.min(channel_data,axis=-1)

        channel_max = np.max(channel_data,axis=-1)

        channel_argmin = np.argmin(channel_data,axis=-1)

        channel_argmax = np.argmax(channel_data,axis=-1)

        channel_rms= np.sqrt(np.mean(channel_data ** 2,axis=-1))  #root mean sqaured #type: ignore


        time_features +=[channel_mean,channel_std,channel_kurtosis,channel_ptp,channel_var,channel_min,channel_max,channel_argmin,channel_argmax,channel_rms]

    return time_features


def save_to_csv(data, filename):
    # Convert numpy.float64 objects to regular floats
    data = [[float(item) if isinstance(item, np.float64) else item for item in row] for row in data] # type: ignore

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

In [77]:
def extract_features(patient, file, start=0, end=3599, window=20, seizure_start=0, seizure_end=0):

    
    raw = mne.io.read_raw_edf(f'{patient}/{file}', preload=True)
    temp=start
    print("********************************************")
    print(type(raw))
    # header=[]
    # with open(f'{patient}_features.csv', 'w', newline='') as csvfile:
    #         writer = csv.writer(csvfile)
    #         writer.writerow(header)

    while temp<end:
        all_features = []
        # create a copy of the original object
        raw_copy = raw.copy()

        # apply slicing on the copy
        if (temp+window)<end:
            current_data = raw_copy.crop(tmin=temp, tmax=(temp+window))
        else:
            current_data = raw_copy.crop(tmin=temp, tmax=end)


        # current_data = raw[:,256*temp:256*(temp+window)]
        print("********************************************")
        print(type(raw))
        spectral_features = extract_spectral_features(current_data, freq_bands)
        time_features = extract_time_features(current_data)
        all_features.append(spectral_features + time_features)

        if temp in range(seizure_start,(seizure_end-1)):
            target = 1
        else:
            target = 0
        all_features.append(target)
        temp+=window

        data = [[float(item) if isinstance(item, np.float64) else item for item in all_features]] # type: ignore

        with open(f'{patient}_features.csv', 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(data)
        

In [78]:
# set path to the seizure.csv file
seizure_csv = 'chb03_seizure.csv'

# read the seizure.csv file as a pandas dataframe
seizure_df = pd.read_csv(seizure_csv)

# set path to the chb03 folder
chb_folder = 'CH03'

# get a list of all the files in the chb03 folder with the '.edf' extension
edf_files_list = [f for f in os.listdir(chb_folder) if f.endswith('.edf')]

# loop through each file in the chb03 folder
for file in edf_files_list:
    # check if the file exists in the seizure.csv file
    if file in seizure_df['File Name'].values:
        # if the file exists, get the corresponding start and end times from the CSV file
        row = seizure_df.loc[seizure_df['File Name'] == file]
        seizure_start = row['Start'].values[0]
        seizure_end = row['End'].values[0]
        extract_features(patient=chb_folder ,file=file,start=0,end=3599,window=20,seizure_start=seizure_start,seizure_end=seizure_end)
    else:
        # if the file does not exist, assign both variables a value of zero
        seizure_start = 0
        seizure_end = 0
        extract_features(patient=chb_folder ,file=file,start=0,end=3599,window=20,seizure_start=seizure_start,seizure_end=seizure_end)
    
    # do something with the seizure_start and seizure_end variables
    # for example, print them to the console
    print(f'File: {file}, Seizure Start: {seizure_start}, Seizure End: {seizure_end}')

Extracting EDF parameters from c:\Users\hp\Documents\GitHub\CDSS-Final-project\CH03\chb03_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


C:\Users\hp\AppData\Local\Temp\ipykernel_10652\591783033.py:4: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(f'{patient}/{file}', preload=True)


********************************************
<class 'mne.io.edf.edf.RawEDF'>
********************************************
<class 'mne.io.edf.edf.RawEDF'>
********************************************
<class 'mne.io.edf.edf.RawEDF'>
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
    Using multitaper spectrum estimation with 7 DPSS windows
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous se

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Applying average reference.
Applying a custom ('EEG',) reference.
    Using multitaper spectrum estimation with 7 DPSS windows
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 7 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

********************************************
<class 'mne.io.edf.edf.RawEDF'>
********************************************
<class 'mne.io.edf.edf.RawEDF'>
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
    Using multitaper spectrum estimation with 7 DPSS windows
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 7 Hz

FIR filter parameters
------

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Applying average reference.
Applying a custom ('EEG',) reference.
    Using multitaper spectrum estimation with 7 DPSS windows


KeyboardInterrupt: 

In [ ]:

def read_eeg_files(directory):
    eeg_data = []

    for filename in os.listdir(directory):
        if filename.endswith('.edf'):
            filepath = os.path.join(directory, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            eeg_data.append(raw)

    return eeg_data

def extract_spectral_features(raw, freq_bands):
    print("********************************************")
    print(type(raw))
    spectral_features = []

    raw.filter(1, 120)
    raw.set_eeg_reference('average')
    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    psds, freqs = psd_array_multitaper(raw.get_data()[eeg_picks], sfreq=raw.info['sfreq'], fmin=0.5, fmax=30) # type: ignore

    for band, freq_range in freq_bands.items():
        band_data = raw.copy().filter(freq_range[0], freq_range[1], picks=eeg_picks).get_data()

        if np.isnan(band_data).any():
            print(f"No data available for {band} band.")
            continue

        band_power = np.mean(np.abs(band_data) ** 2, axis=1)
        band_entropy = entropy(np.abs(band_data) ** 2, axis=1)
        band_skewness = skew(band_data, axis=1)

        spectral_features += [band_power, band_entropy, band_skewness]


    return spectral_features

def extract_time_features(raw):
    time_features = []

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
    eeg_data = raw.get_data()[eeg_picks]

    for channel_data in eeg_data:
        channel_mean = np.mean(channel_data)

        channel_std = np.std(channel_data)

        channel_kurtosis = kurtosis(channel_data)
            
        channel_ptp = np.ptp(channel_data,axis=-1)

        channel_var = np.var(channel_data,axis=-1)

        channel_min = np.min(channel_data,axis=-1)

        channel_max = np.max(channel_data,axis=-1)

        channel_argmin = np.argmin(channel_data,axis=-1)

        channel_argmax = np.argmax(channel_data,axis=-1)

        channel_rms= np.sqrt(np.mean(channel_data ** 2,axis=-1))  #root mean sqaured #type: ignore


        time_features +=[channel_mean,channel_std,channel_kurtosis,channel_ptp,channel_var,channel_min,channel_max,channel_argmin,channel_argmax,channel_rms]

    return time_features


def save_to_csv(data, filename):
    # Convert numpy.float64 objects to regular floats
    data = [[float(item) if isinstance(item, np.float64) else item for item in row] for row in data] # type: ignore

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

In [ ]:

# Example usage:
directory = 'CH03'
freq_bands = {'delta': [0.5, 4], 'theta': [4, 7], 'alpha': [7, 15], 'beta': [15, 30]}

eeg_data = read_eeg_files(directory)

all_features = []

for raw in eeg_data:
    print("********************************************")
    print(type(raw))
    spectral_features = extract_spectral_features(raw, freq_bands)
    time_features = extract_time_features(raw)

    # Combine spectral and time features into a single list
    all_features.append(spectral_features + time_features)

# Save features to a CSV file
save_to_csv(all_features, 'features.csv')
